## 라이브러리 로드

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져옵니다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱합니다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러옵니다.
import pandas as pd

## 서울시 코로나19 발생동향
* [코로나19](http://www.seoul.go.kr/coronaV/coronaStatus.do)

## requests 로 html 문서 받아오기

In [2]:
# 크롤링 할 사이트
base_url = "http://www.seoul.go.kr/coronaV/coronaStatus.do"
response = requests.get( base_url )
response

<Response [200]>

In [3]:
# HTTP응답코드가 200일때만 크롤링하기
if response.status_code == 200:
    soup = bs(response.text, 'html.parser')

str(soup)[:1000]

'\n<!DOCTYPE HTML>\n\n<html lang="ko">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport">\n<title>서울시 코로나19</title>\n<meta content="서울시 코로나19 통합정보 사이트로 안전·방역(발생동향, 클린존, 선별진료소, 해외입국자 안내, 일일브리핑, 홍보물&amp;공적 공급마스크, 보도자료, 일일 소식지&amp;대응일지, 신고(응답소)), 생활정보(주요뉴스, 생활경제 지원, 온라인전시&amp;공연, 온라인도서관, 온라인체육관, 시민생활수기, 언론사 팩트체크, 심리지원), 시민 참여(잠시멈춤 캠페인, 온-서울캠페인, 시민제안)등의 정보를 제공하고 있습니다." name="description"/>\n<meta content="website" property="og:type"/>\n<meta content="서울시 코로나19 통합사이트" property="og:title"/>\n<meta content="서울시 코로나19 통합정보 사이트로 안전·방역(발생동향, 클린존, 선별진료소, 해외입국자 안내, 일일브리핑, 홍보물&amp;공적 공급마스크, 보도자료, 일일 소식지&amp;대응일지, 신고(응답소)), 생활정보(주요뉴스, 생활경제 지원, 온라인전시&amp;공연, 온라인도서관, 온라인체육관, 시민생활수기, 언론사 팩트체크, 심리지원), 시민 참여(잠시멈춤 캠페인, 온-서울캠페인, 시민제안)등의 정보를 제공하고 있습니다." property="og:description"/>\n<meta content="https://www.seoul.go.kr/coronaV/coronaStatus.do" property="og:url"/>\n<meta content="http://www.seoul.go

## 서울 확진자 현황

In [4]:
cont_page = soup.select(".datatable-multi-row")[0]
print(len(cont_page))

9


* table 의 구조
<table>
    <tr>
        <th>제목1</th><th>제목2</th><th>제목3</th>
    </tr>
    <tr>
        <td>컬럼1</td><td>컬럼2</td><td>컬럼2</td>
    </tr>
</table>

```
<table>
    <tr>
        <th>제목1</th><th>제목2</th><th>제목3</th>
    </tr>
    <tr>
        <td>컬럼1</td><td>컬럼2</td><td>컬럼2</td>
    </tr>
</table>
```

In [5]:
# table의 tr 태그를 읽어옵니다.
trs_ex = cont_page.select("tr")
print(len(trs_ex))
trs_ex[-1]

1171


<tr>
<th scope="row"><p class="corona19_no">1</p></th>
<td data-tit="환자 번호" scope="col">2</td>
<td data-tit="확진일">1.24.</td>
<td data-tit="거주지">강서구</td>
<td data-tit="여행력">중국 우한시</td>
<td data-tit="접촉력">해외 접촉</td>
<td data-tit="조치사항">국립중앙의료원(<b class="status1">퇴원</b>)</td>
</tr>

## 행 데이터 리스트로 만들기

In [6]:
def parse_tr(trs):
    table = []
    for tr in trs:
        tds = tr.select("td")
        if len(tds) > 0 :
            row = []
            # 연번
            number = tr.select("th")[0].get_text()
            row.append(number)
            # 연번 외 데이터
            for td in tds:
                val = td.get_text()
                row.append(val)
            table.append(row)
    return table

parse_tr(trs_ex[:3])

[['1170', '12245', '6.17.', '구로구', '-', '수도권 개척교회 관련', '서남병원'],
 ['1169', '12244', '6.17.', '은평구', '-', '확인 중', '국립중앙의료원']]

In [7]:
table_result = parse_tr(trs_ex)
table_result[-1]

['1', '2', '1.24.', '강서구', '중국 우한시', '해외 접촉', '국립중앙의료원(퇴원)']

In [8]:
table_result[0]

['1170', '12245', '6.17.', '구로구', '-', '수도권 개척교회 관련', '서남병원']

## 컬럼명 만들기

In [9]:
# 컬럼명 만들기
cols = trs_ex[0].select("th")
cols

[<th scope="col">연번</th>,
 <th scope="col">환자</th>,
 <th scope="col">확진일</th>,
 <th scope="col">거주지</th>,
 <th scope="col">여행력</th>,
 <th scope="col">접촉력</th>,
 <th scope="col">조치사항</th>]

In [10]:
col_name = []
for col in cols:
    col_name.append(col.get_text())
col_name

['연번', '환자', '확진일', '거주지', '여행력', '접촉력', '조치사항']

## 데이터프레임으로 만들기

In [11]:
df = pd.DataFrame(table_result, columns=col_name)
print(df.shape)
df.head()

(1170, 7)


,연번,환자,확진일,거주지,여행력,접촉력,조치사항
0,1170,12245,6.17.,구로구,-,수도권 개척교회 관련,서남병원
1,1169,12244,6.17.,은평구,-,확인 중,국립중앙의료원
2,1168,12246,6.17.,은평구,-,양천구 운동시설 관련,서남병원
3,1167,12242,6.17.,관악구,-,구로구 확진자 접촉,보라매병원
4,1166,12251,6.17.,수원시,-,확인 중,타시도 이관


## CSV 파일로 저장하기

In [12]:
import datetime

today = datetime.datetime.today()
now = today.strftime('%Y-%m-%d')
now

'2020-06-18'

In [13]:
df.to_csv(f"covid-19-seoul-{now}.csv", index=False)

## CSV 파일 읽어오기

In [14]:
pd.read_csv(f"covid-19-seoul-{now}.csv").head(10)

,연번,환자,확진일,거주지,여행력,접촉력,조치사항
0,1170,12245,6.17.,구로구,-,수도권 개척교회 관련,서남병원
1,1169,12244,6.17.,은평구,-,확인 중,국립중앙의료원
2,1168,12246,6.17.,은평구,-,양천구 운동시설 관련,서남병원
3,1167,12242,6.17.,관악구,-,구로구 확진자 접촉,보라매병원
4,1166,12251,6.17.,수원시,-,확인 중,타시도 이관
5,1165,12240,6.17.,서초구,-,서초구 확진자 접촉,서남병원
6,1164,12229,6.17.,강서구,-,확인 중,은평성모병원
7,1163,12255,6.17.,영등포구,-,확인 중,서울의료원
8,1162,12221,6.17.,도봉구,-,요양시설 관련,서남병원
9,1161,12214,6.17.,주소불명,-,리치웨이 관련,서울백병원


In [15]:
df["접촉력"].value_counts()

해외 접촉 추정       268
이태원 클럽 관련      139
리치웨이 관련         94
확인 중            75
구로구 콜센터 관련      60
              ... 
#8138 접촉         1
#9020 접촉         1
#8709 접촉 추정      1
#5 접촉            1
#935 접촉자         1
Name: 접촉력, Length: 112, dtype: int64

In [16]:
df["조치사항"].value_counts()

서울의료원(퇴원)     227
서울의료원         142
보라매병원(퇴원)     140
서남병원          110
서남병원(퇴원)      108
             ... 
서울의료원(사망)       1
세브란스병원          1
삼성서울병원(퇴원)      1
서울백병원           1
세브란스병원(퇴원)      1
Name: 조치사항, Length: 67, dtype: int64